# Basics

## Initialisierung

In [1]:
import pandas as pd
from datetime import date, datetime
from sklearn import preprocessing

In [2]:
df = pd.DataFrame({"nam": ["Pers A", "Pers B", "Pers C"], "gebdat": [datetime(1974, 1, 1), datetime(1976, 1, 1), datetime(1979, 1, 1)], "stud": ["wahr", "falsch", "wahr"]})
df

,nam,gebdat,stud
0,Pers A,1974-01-01,wahr
1,Pers B,1976-01-01,falsch
2,Pers C,1979-01-01,wahr


## Projektionen
### Spalten umbenennen

In [3]:
df = df.rename(columns={"nam": "Name", "gebdat": "Geburtstag", "stud": "Student"})
df

,Name,Geburtstag,Student
0,Pers A,1974-01-01,wahr
1,Pers B,1976-01-01,falsch
2,Pers C,1979-01-01,wahr


### Spalte hinzufügen

In [4]:
today = date.today() 
def calculateAge(birthDate): 
    return today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day))

df["Alter"] = df.Geburtstag.apply(calculateAge)
df

,Name,Geburtstag,Student,Alter
0,Pers A,1974-01-01,wahr,46
1,Pers B,1976-01-01,falsch,44
2,Pers C,1979-01-01,wahr,41


### Spalte entfernen

In [5]:
df = df.drop(columns=["Alter"])
df

,Name,Geburtstag,Student
0,Pers A,1974-01-01,wahr
1,Pers B,1976-01-01,falsch
2,Pers C,1979-01-01,wahr


## Wertkonvertierung

In [6]:
map = { "wahr": 1, "falsch": 0 }
df["Student"] = df["Student"].apply(lambda x: map[x])
df


,Name,Geburtstag,Student
0,Pers A,1974-01-01,1
1,Pers B,1976-01-01,0
2,Pers C,1979-01-01,1


## Daten kombinieren

In [7]:
df2 = pd.DataFrame({"Name": ["Pers A", "Pers B", "Pers D"], "Ort": ["Bruneck", "Brixen", "Kufstein"]})
df2

,Name,Ort
0,Pers A,Bruneck
1,Pers B,Brixen
2,Pers D,Kufstein


### Inner Join

In [8]:
pd.merge(df, df2, left_on="Name", right_on="Name")

,Name,Geburtstag,Student,Ort
0,Pers A,1974-01-01,1,Bruneck
1,Pers B,1976-01-01,0,Brixen


### Left outer join

In [9]:
pd.merge(df, df2, on="Name", how="left")

,Name,Geburtstag,Student,Ort
0,Pers A,1974-01-01,1,Bruneck
1,Pers B,1976-01-01,0,Brixen
2,Pers C,1979-01-01,1,NaN


## Pivot / Unpivot

### Initialisierung

In [10]:
df = pd.DataFrame({"Person": ["A", "A", "B", "C", "C"], "Art": ["Festnetz", "Mobil", "Mobil","Festnetz", "Fax"], "Telefon": ["111", "222", "333", "444", "555"]})
df

,Person,Art,Telefon
0,A,Festnetz,111
1,A,Mobil,222
2,B,Mobil,333
3,C,Festnetz,444
4,C,Fax,555


### Pivot

In [11]:
dfPiv = df.pivot(index='Person', columns='Art', values='Telefon')
dfPiv

Art,Fax,Festnetz,Mobil
Person,,,
A,NaN,111,222
B,NaN,NaN,333
C,555,444,NaN


In [12]:
dfPiv = dfPiv.reset_index()
dfPiv

Art,Person,Fax,Festnetz,Mobil
0,A,NaN,111,222
1,B,NaN,NaN,333
2,C,555,444,NaN


### Unpivot

In [13]:
dfUnpiv = pd.melt(dfPiv, id_vars=["Person"], var_name="ArtTelefon", value_name="Telefon")
dfUnpiv


,Person,ArtTelefon,Telefon
0,A,Fax,NaN
1,B,Fax,NaN
2,C,Fax,555
3,A,Festnetz,111
4,B,Festnetz,NaN
5,C,Festnetz,444
6,A,Mobil,222
7,B,Mobil,333
8,C,Mobil,NaN


In [14]:
dfUnpiv.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

,Person,ArtTelefon,Telefon
2,C,Fax,555
3,A,Festnetz,111
5,C,Festnetz,444
6,A,Mobil,222
7,B,Mobil,333


## OneHotEncoding

In [15]:
pd.get_dummies(df, columns=["Art"]).head()

,Person,Telefon,Art_Fax,Art_Festnetz,Art_Mobil
0,A,111,0,1,0
1,A,222,0,0,1
2,B,333,0,0,1
3,C,444,0,1,0
4,C,555,1,0,0
